<a href="https://colab.research.google.com/github/hijera/NN_colab_hw/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_%D1%82%D0%B5%D0%BC%D0%B5_%22%D0%93%D0%B8%D0%B1%D1%80%D0%B8%D0%B4%D0%BD%D1%8B%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B%22_(%D0%92%D0%BB%D0%B0%D1%81%D0%BE%D0%B2_%D0%90%D0%BD%D1%82%D0%BE%D0%BD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.  Датасет ml-latest
2.  Вспомнить подходы, которые мы разбирали
3.  Выбрать понравившийся подход к гибридным системам
4.  Написать свою

In [ ]:
!pip install surprise

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
tqdm.pandas()
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline

#Импорт surprise
from surprise import KNNWithMeans, KNNBasic,SVD,BaselineOnly
from surprise import NormalPredictor,KNNWithZScore,SVDpp,NMF,SlopeOne,CoClustering
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate


%matplotlib inline

In [ ]:
!wget  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip -O ml-latest.zip
#!wget https://files.grouplens.org/datasets/movielens/ml-latest.zip -O ml-latest.zip
!unzip -o ml-latest.zip
#!rm ./ml-latest -Rf
#!mv ./ml-latest-small ./ml-latest
prefix = 'ml-latest-small'

--2023-12-21 04:30:19--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest.zip’

ml-latest.zip       100%[===================>] 955.28K  4.66MB/s    in 0.2s    

2023-12-21 04:30:20 (4.66 MB/s) - ‘ml-latest.zip’ saved [978202/978202]

Archive:  ml-latest.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))
tags = pd.read_csv(os.path.join(prefix, 'tags.csv')).fillna("")

In [ ]:
ratings=ratings.drop(columns=['userId'])
ratings.head(10)

,movieId,rating,timestamp
0,1,4.0,964982703
1,3,4.0,964981247
2,6,4.0,964982224
3,47,5.0,964983815
4,50,5.0,964982931
5,70,3.0,964982400
6,101,5.0,964980868
7,110,4.0,964982176
8,151,5.0,964984041
9,157,5.0,964984100


In [ ]:
tags=tags.drop(['userId'],axis=1)
tags['grouped_tags']=tags.groupby('movieId')['tag'].transform(lambda x: '|'.join(x))
gtags=tags.copy()
gtags=gtags.drop(columns=['tag','timestamp']).drop_duplicates()
gtags

,movieId,grouped_tags
0,60756,funny|Highly quotable|will ferrell|comedy|funn...
3,89774,Boxing story|MMA|Tom Hardy
6,106782,drugs|Leonardo DiCaprio|Martin Scorsese|Stock ...
9,48516,way too long|Leonardo DiCaprio|suspense|twist ...
10,431,Al Pacino|gangster|mafia
...,...,...
3674,1948,British
3676,5694,70mm
3677,6107,World War II
3679,7936,austere


In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

movies['grouped_genres'] = [change_string(g).lower() for g in movies.genres.values]
movies.drop(columns=['genres'])

In [ ]:
movies

,movieId,title,genres,grouped_genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,adventure animation children comedy fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy,adventure children fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,comedy romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,comedy drama romance
4,5,Father of the Bride Part II (1995),Comedy,comedy
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,action animation comedy fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,animation comedy fantasy
9739,193585,Flint (2017),Drama,drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,action animation


In [ ]:
tfidf = TfidfVectorizer()
features = tfidf.fit_transform(movies['grouped_genres'])
columns = [(k, tfidf.vocabulary_[k]) for k in tfidf.vocabulary_]
columns = sorted(columns, key=lambda c: c[1])
columns = [c[0] for c in columns]
features = features.todense()
df_features = pd.DataFrame(features, columns=columns)
df_result = pd.concat((movies, df_features), axis=1).drop([ 'grouped_genres','genres'], axis=1)
#columns.remove('genres')



In [ ]:
df_result

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,Toy Story (1995),0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,193583,No Game No Life: Zero (2017),0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,193585,Flint (2017),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
movies_f=

SyntaxError: ignored